(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Analyzing Hotel Ratings on Tripadvisor

In this homework we will focus on practicing two techniques: web scraping and regression. For the first part, we will get some basic information for each hotel in Boston. Then, we will fit a regression model on this information and try to analyze it.

** Task 1 (30 pts)**

We will scrape the data using Beautiful Soup. For each hotel that our search returns, we will get the information below.

![Information to be scraped](hotel_info.png)

Of course, feel free to collect even more data if you want. 

In [91]:
############################################################################################
# get hotel boxes
############################################################################################
from BeautifulSoup import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import requests
import codecs
import json
from selenium import webdriver

base_url = "http://www.tripadvisor.com"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"
'''
#prepare page
bos_hotel_url = "%s/TypeAheadJson?query=%s%%20%s&action=API" % (base_url, 'Boston', 'Massachusetts')
headers = { 'User-Agent' : user_agent }
response = requests.get(bos_hotel_url, headers=headers)
html = response.text.encode('utf-8')
js = json.loads(html)

#boston page
boston_url = base_url + js['results'][0]['urls'][0]['url']
headers = { 'User-Agent' : user_agent }
response = requests.get(boston_url, headers=headers)
html = response.text.encode('utf-8')
soup = BeautifulSoup(html)
li = soup.find("li", {"class": "hotels twoLines"})
city_url = li.find('a', href = True)
'''
#boston hotel page
boston_hotel_url = base_url + '/Hotels-g60745-Boston_Massachusetts-Hotels.html'
headers = { 'User-Agent' : user_agent }
response = requests.get(boston_hotel_url, headers=headers)
print boston_hotel_url
html = response.text.encode('utf-8')
soup = BeautifulSoup(html)

hotel_box_lst = []
#30 hotels each page
while True:
    hotel_boxes = soup.findAll('div', {'class' :'listing easyClear  p13n_imperfect '})
    for hotel_box in hotel_boxes:
        if hotel_box not in hotel_box_lst:
            hotel_box_lst.append(hotel_box)
        else:
            print 'ERROR, exist box'
    del(hotel_boxes)
    next_page_div = soup.find("div", {"class" : "unified pagination standard_pagination"})
    if next_page_div.find('span', {'class' : 'nav next ui_button disabled'}):
        print 'Reached Last Page\n'
        break
    else:
        # If it is not las page there must be the Next URL
        hrefs = next_page_div.findAll('a', href= True)
        for href in hrefs:
            if href.find(text = True) == 'Next':
                boston_hotel_url = base_url + href['href']
        response = requests.get(boston_hotel_url, headers=headers)
        print boston_hotel_url
        html = response.text.encode('utf-8')
        soup = BeautifulSoup(html)


http://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html
http://www.tripadvisor.com/Hotels-g60745-oa30-Boston_Massachusetts-Hotels.html#ACCOM_OVERVIEW
http://www.tripadvisor.com/Hotels-g60745-oa60-Boston_Massachusetts-Hotels.html#ACCOM_OVERVIEW
Reached Last Page



In [93]:
############################################################################################
# function to scrape omni parker house rating summary
############################################################################################
import json
import time

def get_omni_summary(hotel_box):
    start_time = time.time()
    review_url = hotel_box.find('div', {'class' : 'listing_title'}).find(href=True)['href']
    review_url = base_url + review_url
    
    #get review page
    response = requests.get(review_url, headers=headers)
    html = response.text.encode('utf-8')
    soup = BeautifulSoup(html)
    first_review_url = base_url + soup.find('div', {'class': 'reviewSelector   track_back'}).find(href=True)['href']
    print first_review_url
    rating_score = {
        'Omni Parker House:Value:1':0,
        'Omni Parker House:Value:2':0,
        'Omni Parker House:Value:3':0,
        'Omni Parker House:Value:4':0,
        'Omni Parker House:Value:5':0,
        'Omni Parker House:Location:1':0,
        'Omni Parker House:Location:2':0,
        'Omni Parker House:Location:3':0,
        'Omni Parker House:Location:4':0,
        'Omni Parker House:Location:5':0,
        'Omni Parker House:Sleep Quality:1':0,
        'Omni Parker House:Sleep Quality:2':0,
        'Omni Parker House:Sleep Quality:3':0,
        'Omni Parker House:Sleep Quality:4':0,
        'Omni Parker House:Sleep Quality:5':0,
        'Omni Parker House:Rooms:1':0,
        'Omni Parker House:Rooms:2':0,
        'Omni Parker House:Rooms:3':0,
        'Omni Parker House:Rooms:4':0,
        'Omni Parker House:Rooms:5':0,
        'Omni Parker House:Cleanliness:1':0,
        'Omni Parker House:Cleanliness:2':0,
        'Omni Parker House:Cleanliness:3':0,
        'Omni Parker House:Cleanliness:4':0,
        'Omni Parker House:Cleanliness:5':0,
        'Omni Parker House:Service:1':0,
        'Omni Parker House:Service:2':0,
        'Omni Parker House:Service:3':0,
        'Omni Parker House:Service:4':0,
        'Omni Parker House:Service:5':0
    }
    #go to first review
    driver = webdriver.Firefox()
    driver.get(first_review_url)
    main_window_handle = driver.current_window_handle
    #scrape data
    while True:
        review_lst = driver.find_elements_by_class_name('  reviewSelector ')
        for review in review_lst:
            scores = review.find_elements_by_class_name('recommend-answer')
            for score in scores:
                item = 'Omni Parker House:' + \
                score.find_element_by_class_name('recommend-description').text + \
                ':' + \
                score.find_element_by_css_selector('.sprite-rating_ss_fill.rating_ss_fill').get_attribute('alt')[0]
                try:
                    #key exist, ++
                    rating_score[item] = rating_score[item] + 1
                except:
                    #key does not exist, add a new key to dict
                    rating_score[item] = 1
        #close all pop out windows
        for handle in driver.window_handles:
            if handle != main_window_handle:
                driver.switch_to_window(handle)
                driver.close()
        #back to main window
        driver.switch_to_window(main_window_handle)
        try:
            #close login window
            driver.find_element_by_class_name('xCloseGreen').click()
        except:
            pass
        #go to next page
        try:
            driver.find_element_by_css_selector('.nav.next.rndBtn.ui_button.primary.taLnk').click()
        except:
            print 'reached last page\n'
            break
    file = open('./Omni_Parker_House_Score.txt', 'w')
    file.writelines(json.dumps(rating_score))
    file.close()
    print time.time() - start_time

In [94]:
############################################################################################
# function to scrape travel_rating and travel_type for all hotels
############################################################################################
def parse_hotel_info(hotel_box, travel_rating, travel_type):
    traveler_rating_lst = {}
    traveler_type_lst = {}
    start_time = time.time()
    review_url = hotel_box.find('div', {'class' : 'listing_title'}).find(href=True)['href']
    review_url = base_url + review_url
    print review_url
    #get review page
    response = requests.get(review_url, headers=headers)
    html = response.text.encode('utf-8')
    soup = BeautifulSoup(html)
    
    #get hotel name
    hotel_name = soup.find('h1', {'id':'HEADING'}).text
    print hotel_name
    
    #get traveler rating
    traveler_rating_box = soup.find('div', {'class':'col rating '})
    lis = traveler_rating_box.findAll('li')
    for i in range(0, len(lis)):
        key = lis[i].find('div').text
        val = lis[i].findAll('span')[3].text
        traveler_rating_lst[key] = int(val.replace(',', ''))
    
    #get traveler type
    traveler_type_box = soup.find('div', {'class':'col segment '})
    lis = traveler_type_box.findAll('li')
    #lis[i].find('label').find('span').text.strip('(').strip(')')
    for i in range(0, len(lis)):
        traveler_type = lis[i].text
        if 'Families' in traveler_type:
            key = 'Families'
        elif 'Couples' in traveler_type:
            key = 'Couples'
        elif 'Solo' in traveler_type:
            key = 'Solo'
        elif 'Business' in traveler_type:
            key = 'Business'
        elif 'Friends' in traveler_type:
            key = 'Friends'
        val = lis[i].find('label').find('span').text.strip('(').strip(')').replace(',', '')
        traveler_type_lst[key] = int(val)
    
    #get other info & save into dict
    travel_rating[hotel_name] = traveler_rating_lst
    travel_type[hotel_name] = traveler_type_lst

In [96]:
#scrape data
travel_rating = {}
travel_type = {}
for hotel_box in hotel_box_lst:
    if hotel_box.find('div', {'class' :'listing_title'}).find(text=True) == 'Omni Parker House':
        ############################################################################################
        # this is for omni parker house, if not testing this function,
        # please comment this line and leave pass only.  It takes about 35 mins to run
        ############################################################################################
        rating_score = get_omni_summary(hotel_box)
        pass
    else:
        parse_hotel_info(hotel_box, travel_rating, travel_type)
file = open('./travel_rating.txt', 'w')
file.writelines(json.dumps(travel_rating))
file.close()
file = open('./travel_type.txt', 'w')
file.writelines(json.dumps(travel_type))
file.close()

print len(travel_rating)

http://www.tripadvisor.com/Hotel_Review-g60745-d89600-Reviews-Revere_Hotel_Boston_Common-Boston_Massachusetts.html
Revere Hotel Boston Common
http://www.tripadvisor.com/Hotel_Review-g60745-d258705-Reviews-Hotel_Commonwealth-Boston_Massachusetts.html
Hotel Commonwealth
http://www.tripadvisor.com/Hotel_Review-g60745-d94330-Reviews-Seaport_Boston_Hotel-Boston_Massachusetts.html
Seaport Boston Hotel
http://www.tripadvisor.com/Hotel_Review-g60745-d89617-Reviews-The_Westin_Copley_Place-Boston_Massachusetts.html
The Westin Copley Place
http://www.tripadvisor.com/Hotel_Review-g60745-d89620-Reviews-Hyatt_Regency_Boston_Harbor-Boston_Massachusetts.html
Hyatt Regency Boston Harbor
http://www.tripadvisor.com/Hotel_Review-g60745-d313573-Reviews-Courtyard_by_Marriott_Boston_Copley_Square-Boston_Massachusetts.html
Courtyard by Marriott Boston Copley Square
http://www.tripadvisor.com/Hotel_Review-g60745-d89602-Reviews-Sheraton_Boston_Hotel-Boston_Massachusetts.html
Sheraton Boston Hotel
http://www.tri

** Task 2 (20 pts) **

Now, we will use regression to analyze this information. First, we will fit a linear regression model that predicts the average rating. For example, for the hotel above, the average rating is

$$ \text{AVG_SCORE} = \frac{1*31 + 2*33 + 3*98 + 4*504 + 5*1861}{2527}$$

Use the model to analyze the important factors that decide the $\text{AVG_SCORE}$.

In [101]:
############################################################################################
# get avg_score
############################################################################################
import numpy as np
import pandas as pd
import csv
import json

#    read json file
travel_rating_file = open('./travel_rating.txt', 'r')
travel_type_file = open('./travel_type.txt', 'r')
travel_rating = json.loads(travel_rating_file.read())
travel_type = json.loads(travel_type_file.read())
hotel_name_lst = travel_type.keys()

#    calc avg score
avg_score = []
for hotel_name in hotel_name_lst:
    temp_dict = travel_rating[hotel_name]
    nom = temp_dict['Excellent'] * 5 + \
        temp_dict['Very good'] * 4 + \
        temp_dict['Average'] * 3 + \
        temp_dict['Poor'] * 2 + \
        temp_dict['Terrible']
    denom = temp_dict['Excellent'] + \
        temp_dict['Very good'] + \
        temp_dict['Average'] + \
        temp_dict['Poor'] + \
        temp_dict['Terrible']
                  
    if(denom != 0):
        avg_score.append(float(nom) / float(denom))
    else:
        avg_score.append(0.0)

#parse rating summary.dat
'''
rating_summary_dict = 
{
    'Boston Harbor Hotel': {
        Service[totalPoints, count],
        Cleanliness[totalPoints, count],
        ...
    },
    'Boston Hotel Buckminster': {
        Service[totalPoints, count],
        Cleanliness[totalPoints, count],
        ...
    }
}
'''
rating_summary_file = open('./rating-summary.dat', 'r')
rating_summary_dict = {}
for line in rating_summary_file:
    #['Boston Harbor Hotel', 'Service', '1', '11']
    temp_lst = line.split(':')
    if temp_lst[0] not in rating_summary_dict.keys():
        #init for hotel
        rating_summary_dict[temp_lst[0]] = {
            'Service': [0.0, 0.0],
            'Cleanliness': [0.0, 0.0],
            'Value': [0.0, 0.0],
            'Sleep Quality': [0.0, 0.0],
            'Location':[0.0, 0.0],
            'Rooms':[0.0, 0.0],
            'Check in / front desk':[0.0, 0.0],
            'Business service (e.g., internet access)':[0.0, 0.0]
        }
    rating_summary_dict[temp_lst[0]][temp_lst[1]][0] += \
        float(temp_lst[2]) * float(temp_lst[3])
    rating_summary_dict[temp_lst[0]][temp_lst[1]][1] += \
        float(temp_lst[3])

rating_summary_lst = [[], [], [], [], [], [], [], []]
#    calc 8 dimension of Xs
'''
Hilton Boston Downtown / Faneuil Hall => Hilton Boston Downtown  Faneuil Hall
'''
i = 0
for hotel_name in hotel_name_lst:
    i = i + 1
    hotel_name = hotel_name.replace('/', '')
    #'Service'
    if rating_summary_dict[hotel_name]['Service'][1] != 0:
        rating_summary_lst[0].append(
            rating_summary_dict[hotel_name]['Service'][0] /\
            rating_summary_dict[hotel_name]['Service'][1]
        )
    else:
        rating_summary_lst[0].append(0.0)
    #'Cleanliness'
    if rating_summary_dict[hotel_name]['Cleanliness'][1] != 0:
        rating_summary_lst[1].append(
            rating_summary_dict[hotel_name]['Cleanliness'][0] /\
            rating_summary_dict[hotel_name]['Cleanliness'][1]
        )
    else:
        rating_summary_lst[1].append(0.0)
    #'Value'
    if rating_summary_dict[hotel_name]['Value'][1] != 0:
        rating_summary_lst[2].append(
            rating_summary_dict[hotel_name]['Value'][0] /\
            rating_summary_dict[hotel_name]['Value'][1]
        )
    else:
        rating_summary_lst[2].append(0.0)
    #'Sleep Quality'
    if rating_summary_dict[hotel_name]['Sleep Quality'][1] != 0:
        rating_summary_lst[3].append(
            rating_summary_dict[hotel_name]['Sleep Quality'][0] /\
            rating_summary_dict[hotel_name]['Sleep Quality'][1]
        )
    else:
        rating_summary_lst[3].append(0.0)
    #'Location'
    if rating_summary_dict[hotel_name]['Location'][1] != 0:
        rating_summary_lst[4].append(
            rating_summary_dict[hotel_name]['Location'][0] /\
            rating_summary_dict[hotel_name]['Location'][1]
        )
    else:
        rating_summary_lst[4].append(0.0)
    #'Rooms'
    if rating_summary_dict[hotel_name]['Rooms'][1] != 0:
        rating_summary_lst[5].append(
            rating_summary_dict[hotel_name]['Rooms'][0] /\
            rating_summary_dict[hotel_name]['Rooms'][1]
        )
    else:
        rating_summary_lst[5].append(0.0)
    #'Check in / front desk'
    if rating_summary_dict[hotel_name]['Check in / front desk'][1] != 0:
        rating_summary_lst[6].append(
            rating_summary_dict[hotel_name]['Check in / front desk'][0] /\
            rating_summary_dict[hotel_name]['Check in / front desk'][1]
        )
    else:
        rating_summary_lst[6].append(0.0)
    #'Business service (e.g., internet access)'
    if rating_summary_dict[hotel_name]['Business service (e.g., internet access)'][1] != 0:
        rating_summary_lst[7].append(
            rating_summary_dict[hotel_name]['Business service (e.g., internet access)'][0] /\
            rating_summary_dict[hotel_name]['Business service (e.g., internet access)'][1]
        )
    else:
        rating_summary_lst[7].append(0.0)
#fit into 8 dimension 
avg_score = np.array(avg_score)
rating_summary_lst = np.array(rating_summary_lst).T

In [102]:
############################################################################################
# put ratting_summary & avg_score to csv file
############################################################################################

columns = ['Service', 'Cleanliness', 'Value', 'Sleep Quality', 
           'Location', 'Rooms', 'Check in / front desk', 'Business service (e.g., internet access)']
os.system('rm ./ratting_summary.csv')
os.system('touch ./ratting_summary.csv')
dataframe_file = open('./ratting_summary.csv', 'r+b')
writer = csv.writer(dataframe_file, delimiter=',')
writer.writerow(columns)
for line in rating_summary_lst:
    writer.writerow([line[0], line[1], line[2], line[3], 
                    line[4], line[5], line[6], line[7]])

columns = ['avg_score']
os.system('rm ./avg_score.csv')
os.system('touch ./avg_score.csv')
avg_score_file = open('./avg_score.csv', 'r+b')
writer = csv.writer(avg_score_file, delimiter=',')
writer.writerow(columns)
for line in avg_score:
    writer.writerow([line])
    
dataframe_file.close()
avg_score_file.close()

In [103]:
############################################################################################
# fit into linear regression
############################################################################################
import statsmodels.api as sm
import numpy as np

df = pd.read_csv('./ratting_summary.csv')
avg_score_df = pd.read_csv('./avg_score.csv')
columns = df.columns
model = sm.OLS(avg_score_df, df)
results = model.fit()
print results.summary()
print "Confidence Intervals:", results.conf_int()
print "Parameters:", results.params

                            OLS Regression Results                            
Dep. Variable:              avg_score   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.648e+04
Date:                Tue, 29 Mar 2016   Prob (F-statistic):          5.77e-103
Time:                        14:48:57   Log-Likelihood:                 70.546
No. Observations:                  72   AIC:                            -125.1
Df Residuals:                      64   BIC:                            -106.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------------------

** Task 3 (30 pts) **

Finally, we will use logistic regression to decide if a hotel is _excellent_ or not. We classify a hotel as _excellent_ if more than **60%** of its ratings are 5 stars. This is a binary attribute on which we can fit a logistic regression model. As before, use the model to analyze the data.

In [ ]:
############################################################################################
# scrape row bar from website and calculate excellent
############################################################################################
excellent_dict = {}
def excellent_hotel(hotel_box, excellent_dict):
    review_url = hotel_box.find('div', {'class' : 'listing_title'}).find(href=True)['href']
    review_url = base_url + review_url
    
    #get review page
    response = requests.get(review_url, headers=headers)
    html = response.text.encode('utf-8')
    soup = BeautifulSoup(html)
    
    excellent_li = soup.find('div', {'class': 'col rating '}).find('li')
    percentage = float(excellent_li.find('span', {'class': 'row_fill'}).get('style').split(':')[1].replace('%;', '')) / 100.0
    hotel_name = soup.find('h1', {'id':'HEADING'}).text
    if percentage > 0.6:
        excellent_dict[hotel_name] = True
    else:
        excellent_dict[hotel_name] = False

    
for hotel_box in hotel_box_lst:
    excellent_hotel(hotel_box, excellent_dict)

In [35]:
############################################################################################
# put excellent list into csv file
############################################################################################
excellent_lst = []
for hotel_name in hotel_name_lst:
    excellent_lst.append(excellent_dict[hotel_name])

os.system('rm ./excellent.csv')    
os.system('touch ./excellent.csv')
excellent_file = open('./excellent.csv', 'r+b')
columns = ['excellent']
writer = csv.writer(excellent_file, delimiter=',')
writer.writerow(columns)
for line in excellent_lst:
    writer.writerow([line])
    
excellent_file.close()

In [36]:
############################################################################################
# fit into logical regression
############################################################################################
excellent_df = pd.read_csv('./excellent.csv')
logit = sm.Logit(excellent_df, df)
 
# fit the model
result = logit.fit()
print result.summary()

Optimization terminated successfully.
         Current function value: 0.313891
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:              excellent   No. Observations:                   69
Model:                          Logit   Df Residuals:                       61
Method:                           MLE   Df Model:                            7
Date:                Tue, 29 Mar 2016   Pseudo R-squ.:                  0.4378
Time:                        13:37:05   Log-Likelihood:                -21.658
converged:                       True   LL-Null:                       -38.524
                                        LLR p-value:                 1.934e-05
                                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------------
Service                                      7.9

-------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()